In [ ]:
###Passo 1: Instalar as bibliotecas e importar os dados

# Importar as bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile

# Definir um estilo para os gráficos
sns.set_style("whitegrid")

# Carregar os datasets (ajuste o caminho se necessário)
try:
    customers = pd.read_csv('leonardo-nunes-260/analise-vendas-ecommerce-brasil/notebooks/data/olist_customers_dataset.csv', compression='zip')
    orders = pd.read_csv('leonardo-nunes-260/analise-vendas-ecommerce-brasil/notebooks/data/olist_orders_dataset.csv', compression='zip')
    order_items = pd.read_csv('leonardo-nunes-260/analise-vendas-ecommerce-brasil/notebooks/data/olist_order_items_dataset.csv', compression='zip')
    products = pd.read_csv('leonardo-nunes-260/analise-vendas-ecommerce-brasil/notebooks/data/olist_products_dataset.csv', compression='zip')
    translation = pd.read_csv('leonardo-nunes-260/analise-vendas-ecommerce-brasil/notebooks/data/product_category_name_translation.csv')

except FileNotFoundError as e:
    print(f"Erro ao carregar os arquivos: {e}")
    print("Verifique se a estrutura de pastas está correta: o script na pasta 'notebooks' e os dados na pasta 'data'.")
    exit()


# Unir os dataframes para criar um dataset único e mais rico
# Merge 1: orders com customers
df = pd.merge(orders, customers, on='customer_id')

# Merge 2: df com order_items
df = pd.merge(df, order_items, on='order_id')

# Merge 3: df com products
df = pd.merge(df, products, on='product_id')

# Merge 4: df com a tradução das categorias
df = pd.merge(df, translation, on='product_category_name')

print("Dados carregados e unificados com sucesso!")
print("Shape do dataframe final:", df.shape)
df.head()

Certifique-se de que os arquivos .csv estão na mesma pasta do seu script/notebook.


NameError: name 'orders' is not defined

: 

In [ ]:
###Passo 2: Limpeza e Preparação dos Dados

# Vamos selecionar apenas as colunas que nos interessam para a análise
colunas_relevantes = [
    'order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
    'price', 'freight_value', 'product_category_name_english',
    'customer_unique_id', 'customer_city', 'customer_state'
]
df_limpo = df[colunas_relevantes]

# Verificar valores nulos
print("Valores nulos por coluna:")
print(df_limpo.isnull().sum())

# Converter a coluna de data para o formato datetime
df_limpo['order_purchase_timestamp'] = pd.to_datetime(df_limpo['order_purchase_timestamp'])

# Vamos focar apenas em pedidos entregues ('delivered') para garantir a análise de vendas concretizadas
df_limpo = df_limpo[df_limpo['order_status'] == 'delivered']

print("\nDados limpos e preparados.")
df_limpo.info()

NameError: name 'df' is not defined

In [ ]:
###Passo 3: Análise Exploratória de Dados (EDA)

In [ ]:
##Pergunta 1: Em quais estados devemos focar?

# Contar o número de pedidos por estado
pedidos_por_estado = df_limpo['customer_state'].value_counts().reset_index()
pedidos_por_estado.columns = ['Estado', 'Total de Pedidos']

# Visualização
plt.figure(figsize=(15, 8))
sns.barplot(x='Estado', y='Total de Pedidos', data=pedidos_por_estado.head(10))
plt.title('Top 10 Estados por Número de Pedidos', fontsize=16)
plt.xlabel('Estado', fontsize=12)
plt.ylabel('Número de Pedidos', fontsize=12)
plt.xticks(rotation=45)
plt.show()

#Insight: A análise mostrará uma concentração massiva em São Paulo (SP), seguido por Rio de Janeiro (RJ) e Minas Gerais (MG). Isso indica onde a base de clientes é mais forte.

NameError: name 'df_limpo' is not defined

In [ ]:
##Pergunta 2: Quais as categorias de produtos mais vendidas?

# Contar o número de vendas por categoria de produto
vendas_por_categoria = df_limpo['product_category_name_english'].value_counts().reset_index()
vendas_por_categoria.columns = ['Categoria', 'Total de Vendas']

# Visualização
plt.figure(figsize=(15, 8))
sns.barplot(x='Total de Vendas', y='Categoria', data=vendas_por_categoria.head(10), orient='h')
plt.title('Top 10 Categorias de Produtos Mais Vendidas', fontsize=16)
plt.xlabel('Número de Vendas', fontsize=12)
plt.ylabel('Categoria', fontsize=12)
plt.show()

#Insight: Categorias como "bed_bath_table", "health_beauty" e "sports_leisure" provavelmente aparecerão no topo. Isso informa ao marketing quais produtos promover.

In [ ]:
##Pergunta 3: Qual o faturamento por categoria?

# Calcular o faturamento por categoria (preço do produto)
faturamento_por_categoria = df_limpo.groupby('product_category_name_english')['price'].sum().sort_values(ascending=False).reset_index()
faturamento_por_categoria.columns = ['Categoria', 'Faturamento Total']

# Visualização
plt.figure(figsize=(15, 8))
sns.barplot(x='Faturamento Total', y='Categoria', data=faturamento_por_categoria.head(10), orient='h')
plt.title('Top 10 Categorias por Faturamento', fontsize=16)
plt.xlabel('Faturamento (em R$)', fontsize=12)
plt.ylabel('Categoria', fontsize=12)
plt.show()

#Insight: Esta visão complementa a anterior. Uma categoria pode vender muito, mas ter um faturamento baixo se os produtos forem baratos. Focar em categorias de alto faturamento é crucial para a lucratividade.

In [ ]:
"""

Conclusão da Análise:

A análise dos dados de vendas revelou insights cruciais para a estratégia da empresa:

- Concentração Geográfica: Existe uma forte concentração de vendas na região Sudeste, com o estado de São Paulo (SP) liderando com uma margem expressiva, seguido por Rio de Janeiro (RJ) e Minas Gerais (MG).
- Performance de Categorias: As categorias com maior volume de vendas são 'cama_mesa_banho' e 'beleza_saude'. No entanto, em termos de faturamento, categorias com ticket médio mais alto, como 'relogios_presentes' e 'informatica_acessorios', também se destacam.

Recomendação Estratégica:

Com base nesta análise, recomendamos as seguintes ações:

1) Para o Marketing:

- Ação 1: Intensificar as campanhas de marketing digital geolocalizadas para os estados de SP, RJ e MG, aproveitando a base de clientes já consolidada.
- Ação 2: Criar campanhas de e-mail marketing e anúncios focados nas categorias de 'cama_mesa_banho' e 'beleza_saude' para impulsionar ainda mais o volume de vendas.

2) Para o Estoque:

- Ação 1: Garantir um nível de estoque robusto para as categorias de maior faturamento e volume, como 'cama_mesa_banho', 'beleza_saude' e 'relogios_presentes', especialmente nos centros de distribuição que atendem a região Sudeste, para evitar ruptura de estoque.
- Ação 2: Analisar as categorias com baixo volume de vendas e faturamento (as que ficam no final da lista) para avaliar a possibilidade de descontinuar produtos ou criar promoções de queima de estoque.
"""

"\n\nConclusão da Análise:\n\nA análise dos dados de vendas revelou insights cruciais para a estratégia da empresa:\n\n- Concentração Geográfica: Existe uma forte concentração de vendas na região Sudeste, com o estado de São Paulo (SP) liderando com uma margem expressiva, seguido por Rio de Janeiro (RJ) e Minas Gerais (MG).\n- Performance de Categorias: As categorias com maior volume de vendas são 'cama_mesa_banho' e 'beleza_saude'. No entanto, em termos de faturamento, categorias com ticket médio mais alto, como 'relogios_presentes' e 'informatica_acessorios', também se destacam.\n\nRecomendação Estratégica:\n\nCom base nesta análise, recomendamos as seguintes ações:\n\nPara o Marketing:\n\n- Ação 1: Intensificar as campanhas de marketing digital geolocalizadas para os estados de SP, RJ e MG, aproveitando a base de clientes já consolidada.\n- Ação 2: Criar campanhas de e-mail marketing e anúncios focados nas categorias de 'cama_mesa_banho' e 'beleza_saude' para impulsionar ainda mais 